# Phase 4: Causality Analysis (Granger & VAR)
**Goal:** Establish causal relationship between Fed sentiment and bond market movements.

## Improved Methodology
1. **Time Series Preprocessing**: Handle non-stationarity and seasonality
2. **VAR Model**: Vector Autoregression for multivariate analysis
3. **Granger Causality**: Test for predictive causality
4. **Impulse Response**: Analyze shock propagation
5. **Control Variables**: Include macroeconomic indicators

In [12]:
# ! pip install statsmodels
# ! pip  install arch

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from arch import arch_model
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

In [14]:
# Load Fed Sentiment Data
fed_sentiment = pd.read_csv('../data/processed/fomc_roberta_monthly_index.csv')
fed_sentiment['date'] = pd.to_datetime(fed_sentiment['date'])
fed_sentiment = fed_sentiment.set_index('date').sort_index()

# Load Market Data
start_date = fed_sentiment.index.min() - timedelta(days=30)
end_date = fed_sentiment.index.max() + timedelta(days=30)

# Download TNX (10-year Treasury yield) and MOVE (bond market volatility)
market_data = yf.download(['^TNX', '^MOVE'], start=start_date, end=end_date)
market_data = market_data['Close'].dropna()

# Download control variables
control_data = yf.download(['^VIX', 'DX-Y.NYB'], start=start_date, end=end_date)
control_data = control_data['Close'].dropna()

print(f"Fed sentiment data: {len(fed_sentiment)} observations")
print(f"Market data: {len(market_data)} observations")
print(f"Control data: {len(control_data)} observations")

FileNotFoundError: [Errno 2] No such file or directory: '../data/fomc_roberta_monthly_index.csv'

In [ ]:
# Merge and resample to monthly
combined_data = pd.concat([
    fed_sentiment['sentiment_index'],
    market_data.resample('M').last(),
    control_data.resample('M').last()
], axis=1).dropna()

# Rename columns
combined_data.columns = ['fed_sentiment', 'tnx_yield', 'move_volatility', 'vix', 'dollar_index']

print("Combined dataset shape:", combined_data.shape)
print("\nFirst few rows:")
display(combined_data.head())

# Plot the time series
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
combined_data.plot(subplots=True, ax=axes.flatten()[:5])
plt.tight_layout()
plt.show()

In [ ]:
# Stationarity Test Function
def test_stationarity(series, name):
    result = adfuller(series.dropna())
    print(f'\n{name} Stationarity Test:')
    print(f'ADF Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value:.4f}')
    
    if result[1] < 0.05:
        print("✓ Stationary")
    else:
        print("✗ Non-stationary")
    
    return result[1] < 0.05

# Test stationarity for all variables
stationary_results = {}
for col in combined_data.columns:
    stationary_results[col] = test_stationarity(combined_data[col], col)

# Difference non-stationary series
diff_data = combined_data.copy()
for col, is_stationary in stationary_results.items():
    if not is_stationary:
        diff_data[f'{col}_diff'] = combined_data[col].diff().dropna()
        print(f"\nDifferenced {col}")

print("\nFinal dataset shape:", diff_data.shape)

In [ ]:
# Select variables for VAR (prefer differenced if non-stationary)
var_variables = []
for col in ['fed_sentiment', 'tnx_yield', 'move_volatility', 'vix', 'dollar_index']:
    if stationary_results[col]:
        var_variables.append(col)
    else:
        var_variables.append(f'{col}_diff')

# Drop NA values from differencing
var_data = diff_data[var_variables].dropna()

print("Variables for VAR analysis:", var_variables)
print("VAR dataset shape:", var_data.shape)

# Determine optimal lag length
model = VAR(var_data)
lag_selection = model.select_order(maxlags=12)
print("\nLag order selection:")
print(lag_selection.summary())

# Fit VAR model
optimal_lags = lag_selection.aic
var_model = model.fit(optimal_lags)
print(f"\nFitted VAR({optimal_lags}) model")
print(var_model.summary())

In [ ]:
# Granger Causality Tests
def granger_test(data, x, y, max_lags=8):
    print(f"\n=== Granger Causality: {x} → {y} ===")
    try:
        test_result = grangercausalitytests(data[[y, x]], max_lags, verbose=False)
        
        # Get F-test p-values for different lags
        p_values = []
        for lag in range(1, max_lags+1):
            f_test = test_result[lag][0]['ssr_ftest']
            p_values.append(f_test[1])
        
        min_p = min(p_values)
        best_lag = p_values.index(min_p) + 1
        
        print(f"Minimum p-value: {min_p:.4f} (at lag {best_lag})")
        
        if min_p < 0.05:
            print(f"✓ {x} Granger-causes {y} (significant at 5% level)")
        else:
            print(f"✗ No Granger causality from {x} to {y}")
        
        return min_p, best_lag
    
    except Exception as e:
        print(f"Error in Granger test: {e}")
        return None, None

# Test causality between Fed sentiment and market variables
causality_results = []
for target in ['tnx_yield', 'move_volatility']:
    for source in ['fed_sentiment']:
        p_val, lag = granger_test(var_data, source, target)
        if p_val is not None:
            causality_results.append({
                'source': source,
                'target': target,
                'p_value': p_val,
                'optimal_lag': lag,
                'significant': p_val < 0.05
            })

# Display results
results_df = pd.DataFrame(causality_results)
print("\n=== Summary of Granger Causality Tests ===")
display(results_df)

In [ ]:
# Impulse Response Analysis
def plot_irf(var_model, impulse, response, periods=12):
    irf = var_model.irf(periods)
    
    # Find column indices
    impulse_idx = var_model.names.index(impulse)
    response_idx = var_model.names.index(response)
    
    # Plot impulse response
    plt.figure(figsize=(10, 6))
    irf.plot(impulse=impulse_idx, response=response_idx, 
             orth=True,  # Orthogonalized IRF
             plot_stderr=True)
    plt.title(f'Impulse Response: {impulse} → {response}')
    plt.grid(True, alpha=0.3)
    plt.show()

# Plot IRFs for significant relationships
for result in causality_results:
    if result['significant']:
        impulse = result['source']
        response = result['target']
        print(f"\nPlotting IRF: {impulse} → {response}")
        plot_irf(var_model, impulse, response)

In [ ]:
# Variance Decomposition
def plot_variance_decomp(var_model, periods=12):
    fevd = var_model.fevd(periods)
    
    fig, axes = plt.subplots(len(var_model.names), len(var_model.names), 
                            figsize=(15, 15))
    
    for i in range(len(var_model.names)):
        for j in range(len(var_model.names)):
            if len(var_model.names) == 1:
                ax = axes
            elif len(var_model.names) > 1:
                ax = axes[i, j]
            
            fevd.plot(i, j, ax=ax)
            ax.set_title(f'{var_model.names[j]} explained by {var_model.names[i]}')
    
    plt.tight_layout()
    plt.show()

print("\n=== Forecast Error Variance Decomposition ===")
plot_variance_decomp(var_model)